In [ ]:
from Crypto.Cipher import AES
from Crypto import Random
import base64
import binascii
import numpy as np
import pandas as pd
from IPython.display import HTML
import json

# Web Scrapping
import requests as rq
from bs4 import BeautifulSoup as bfs

In [ ]:
HTML('<iframe src=https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode=5CHP784HD width=1000 height=350></iframe>')

https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode=5CHP784HD

JS: 
```
var key = CryptoJS.enc.Base64.parse("AYf833rD2Q1oM0r6A7h+/g==");
var iv  = CryptoJS.enc.Base64.parse("M6OUM+Td58TdudRQK4kF1A==");
var decrypted = CryptoJS.AES.decrypt(resultEnc, key, {iv: iv});
var result = JSON.parse(decrypted.toString(CryptoJS.enc.Utf8));
```

Results:
```
resultEnc = '3zxaer+64X4MlhwuD6brwEHBNFvFQ1/1v6HvtgY6H8cXvrxxGlWtGdWGRqfdmh5BmOnTTj0AplYL4GXExOqQU8s2quJTqvawdfazWRNL/9y76ldSjVbEl3XaFhl7yzA3uX36uOswsfUEXvQrvXeSWd8+KaVlbzyUDZnlytuBi+GWaipKz6NdSqmY8U3nPdaoVDf7euSP/7Z3L3sLLYLmQ0UgOtp+2+GSDL8cVHNMZTHsBXwP7LOQtXKjOLBJ0wm7svr+omqmh1Hr8kG2Kcj2JTkUQd+mrvbTeofzmzYLWezHJzwMeYMXsx5VdgbJtnaJoi5Jv6MVVJkeQFOJ8t1hYfuQoFfpIrDZLS4pdY8FhQFFOuPELOZuDeAAXwAG8Y2idzpaZIPD1V+4+iisal0Ud0Sl6XyjTqSaz4L9ceYJMKAdo5/kXZWqiBotN/u+rUdmtAaVAoMVA7zhPN7EIvpN+dWfLn/37ukWY1R51m99C5ryH0JoZSA2DmFpziEfqLk6y6jvGEY/EVx3sRiFgh1VsYs+o3VDKlYEsKh5sPoGi9pmQiZOTdGazXFxbCpVJJT3KxU12bleNrQdrABVvxAhh9GgtfWLFdozlpW11K8TbhTheu6Qr8yRBnJpq5gKe5ti'

key = {
    sigBytes: 16,
    words: {
        0: 25689311,
        1: 2059655437,
        2: 1748191994,
        3: 62422782
    }
}

iv = {
    sigBytes: 16,
    words: {
        0: 866358323,
        1: -455219260,
        2: -575024048,
        3: 730400212
    }
}

result = {
    catRank: 43,
    categoryName: "STC Olympic Hommes CA 35-44",
    entryArt: "Bezahlt",
    entryPayart: "Online",
    eventDate: "11.09.2016",
    eventName: "Triathlon Yverdon-les-Bains",
    eventRaceNr: "B628CCC3D9A7DDBF558DA920B9E33C7D"
    overCategoryName: "STC Olympic Hommes Overall",
    overRank: 122,
    provider: "Datasport",
    raceNr: 18594,
    racePayload: "E09137F006F1390053225B57ABFA512DF7BC65269C53D92AC9AB3E2E044E6DFD",
    resultState: "Klassiert",
    runtime: "3:02.32,0",
    startNumber: 166
}
```

In [ ]:
def decrypt_response(response):
    
    html_page = '<p id="crypted" style="display:none;">' + response + '</p>'
    
    jquery_file = open('Lib/jquery.min.js', 'r')
    crypto_js_file = open('Lib/cryptojs-aes.js', 'r')
    
    javascript = '<script type="text/Javascript">' + jquery_file.read() + '</script>'
    javascript += '<script type="text/Javascript">' + crypto_js_file.read() + '</script>'
    
    javascript += """
        <script type="text/Javascript">
            var key = CryptoJS.enc.Base64.parse("AYf833rD2Q1oM0r6A7h+/g==");
            var iv  = CryptoJS.enc.Base64.parse("M6OUM+Td58TdudRQK4kF1A==");

            var crypted = $('#crypted').text();

            var decrypted = CryptoJS.AES.decrypt(crypted, key, {iv: iv});
            var result = decrypted.toString(CryptoJS.enc.Utf8);

            var command = "result = '" + result + "'";

            var kernel = IPython.notebook.kernel;
            kernel.execute(command);

        </script>
    """

    HTML(html_page + javascript)
    return json.loads(result)

In [ ]:
resultEnc = '3zxaer+64X4MlhwuD6brwEHBNFvFQ1/1v6HvtgY6H8cXvrxxGlWtGdWGRqfdmh5BmOnTTj0AplYL4GXExOqQU8s2quJTqvawdfazWRNL/9y76ldSjVbEl3XaFhl7yzA3uX36uOswsfUEXvQrvXeSWd8+KaVlbzyUDZnlytuBi+GWaipKz6NdSqmY8U3nPdaoVDf7euSP/7Z3L3sLLYLmQ0UgOtp+2+GSDL8cVHNMZTHsBXwP7LOQtXKjOLBJ0wm7svr+omqmh1Hr8kG2Kcj2JTkUQd+mrvbTeofzmzYLWezHJzwMeYMXsx5VdgbJtnaJoi5Jv6MVVJkeQFOJ8t1hYfuQoFfpIrDZLS4pdY8FhQFFOuPELOZuDeAAXwAG8Y2idzpaZIPD1V+4+iisal0Ud0Sl6XyjTqSaz4L9ceYJMKAdo5/kXZWqiBotN/u+rUdmtAaVAoMVA7zhPN7EIvpN+dWfLn/37ukWY1R51m99C5ryH0JoZSA2DmFpziEfqLk6y6jvGEY/EVx3sRiFgh1VsYs+o3VDKlYEsKh5sPoGi9pmQiZOTdGazXFxbCpVJJT3KxU12bleNrQdrABVvxAhh9GgtfWLFdozlpW11K8TbhTheu6Qr8yRBnJpq5gKe5ti'
decrypt_response(resultEnc)

Get all data about runners.

In [ ]:
ADRESS_PERSONNAL_PAGE = 'https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode='
FILE_NAME = 'unique_acode_2009_2015.csv'
PATH_TO_DATA = './Data/acode_2009_2015/'

data_acode = pd.read_csv(PATH_TO_DATA+FILE_NAME, sep='\t')
data_acode.head()

In [ ]:
ATTRIBUTE_RUN = 'url_run_event'
for acode in data_acode['acode']:
    url = 'https://www.datasport.com/sys/myds/ajax/getDSInfo.htm?acode='+acode
    print(url)
    filters_page = rq.get(url) 
    page = bfs(filters_page.text, 'html.parser')
    
    # get the birtyear
    matches = page.findAll("p", { "class" : "small" })
    birth_year = matches[1].text.split(" ")[1:][0]
    
    # get information on the table.
    table_run_event = page.find("table", { "class" : "table table-condensed table-striped" })
    information_runner = parse_html_table(table_run_event)
    
    for url_ajax in information_runner[ATTRIBUTE_RUN]:
        #for all url get the response --> decrypt the value --> store information
        #print ('request to : '+url_ajax)
        #ajax_response = rq.get(url_ajax)
        #print(ajax_response.text)       

    break

In [ ]:
def parse_html_table(table):
    URL_WEBSITE = 'https://www.datasport.com'
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    
    
    url_information_runner = []
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        
        columns = row.find_all('td')
        for index, column in enumerate(columns):
            df.iat[row_marker,column_marker] = column.get_text()
            
            # add the url to get informations about the runners.
            if(index == 1):
                url_information_runner.append(URL_WEBSITE+column.find('a')['href'])
                
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    #print(url_information_runner)
    # add columns url
    df['url_run_event'] = pd.Series(url_information_runner)
    return df